<a href="https://colab.research.google.com/github/sergekamanzi/ChatBotpro/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install sentence-transformers -q

from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Load your full banking dataset
df = pd.read_csv("/content/banking.csv")
df = df.dropna(subset=["question", "response"])
questions = df["question"].tolist()
responses = df["response"].tolist()

# Load a semantic sentence transformer (MiniLM is fast & accurate)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode all dataset questions
question_embeddings = model.encode(questions, convert_to_tensor=True)

# Chatbot memory loop
def chatbot():
    print("🔐 Banking Chatbot is ready! Type 'exit' to quit.\n")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Chatbot: Goodbye!")
            break

        # Encode user question
        user_embedding = model.encode(user_input, convert_to_tensor=True)

        # Compute similarity with dataset
        similarity_scores = util.cos_sim(user_embedding, question_embeddings)[0]
        best_match_idx = similarity_scores.argmax().item()
        best_score = similarity_scores[best_match_idx].item()

        # Confidence threshold
        if best_score > 0.6:
            print("Chatbot:", responses[best_match_idx])
        else:
            print("Chatbot: Sorry, I couldn't understand that. Try asking in a different way.")

chatbot()
